In [4]:
import lithops
import ray

In [5]:
def extract_color(obj):
    from colorthief import ColorThief
    body = obj.data_stream
    dominant_color = ColorThief(body).get_color(quality=10)
    return dominant_color, obj.key

@ray.remote
def identify_colorspace(data):
    import colorsys
    color, name =  data.result()

    hsv = colorsys.rgb_to_hsv(color[0], color[1], color[2])
    val = hsv[0] * 180
    return name, val

In [6]:
if __name__ == '__main__':
    import time
    ray.init(ignore_reinit_error=True)
    fexec = lithops.LocalhostExecutor(log_level=None)

    my_data = fexec.map(extract_color, 'cos://gvernikuseast/500flowers/')
    results = [identify_colorspace.remote(d) for d in my_data]

    for res in results:
        value = ray.get(res)
        print("Image: " + value[0] + ", dominant color HSV range: " + str(value[1]))
    ray.shutdown()

2021-05-03 21:21:20,200	INFO services.py:1180 -- View the Ray dashboard at http://127.0.0.1:8265


Image: 500flowers/500flowers/image_0001.jpg, dominant color HSV range: 35.454545454545446
Image: 500flowers/500flowers/image_0002.jpg, dominant color HSV range: 12.0
Image: 500flowers/500flowers/image_0003.jpg, dominant color HSV range: 69.64285714285714
Image: 500flowers/500flowers/image_0004.jpg, dominant color HSV range: 166.95652173913044
Image: 500flowers/500flowers/image_0005.jpg, dominant color HSV range: 38.78048780487804
Image: 500flowers/500flowers/image_0006.jpg, dominant color HSV range: 90.0
Image: 500flowers/500flowers/image_0007.jpg, dominant color HSV range: 108.75
Image: 500flowers/500flowers/image_0008.jpg, dominant color HSV range: 27.796610169491526
Image: 500flowers/500flowers/image_0009.jpg, dominant color HSV range: 23.793103448275865
Image: 500flowers/500flowers/image_0010.jpg, dominant color HSV range: 27.169811320754715
Image: 500flowers/500flowers/image_0011.jpg, dominant color HSV range: 27.692307692307693
Image: 500flowers/500flowers/image_0012.jpg, dominan

Image: 500flowers/500flowers/image_0101.jpg, dominant color HSV range: 40.0
Image: 500flowers/500flowers/image_0102.jpg, dominant color HSV range: 58.46153846153847
Image: 500flowers/500flowers/image_0103.jpg, dominant color HSV range: 81.66666666666667
Image: 500flowers/500flowers/image_0104.jpg, dominant color HSV range: 22.5
Image: 500flowers/500flowers/image_0105.jpg, dominant color HSV range: 50.76923076923078
Image: 500flowers/500flowers/image_0106.jpg, dominant color HSV range: 47.14285714285714
Image: 500flowers/500flowers/image_0107.jpg, dominant color HSV range: 56.66666666666667
Image: 500flowers/500flowers/image_0108.jpg, dominant color HSV range: 23.225806451612904
Image: 500flowers/500flowers/image_0109.jpg, dominant color HSV range: 83.33333333333333
Image: 500flowers/500flowers/image_0110.jpg, dominant color HSV range: 45.882352941176464
Image: 500flowers/500flowers/image_0111.jpg, dominant color HSV range: 90.0
Image: 500flowers/500flowers/image_0112.jpg, dominant colo

Image: 500flowers/500flowers/image_0197.jpg, dominant color HSV range: 26.666666666666664
Image: 500flowers/500flowers/image_0198.jpg, dominant color HSV range: 22.5
Image: 500flowers/500flowers/image_0199.jpg, dominant color HSV range: 25.263157894736842
Image: 500flowers/500flowers/image_0200.jpg, dominant color HSV range: 34.0
Image: 500flowers/500flowers/image_0201.jpg, dominant color HSV range: 43.37349397590362
Image: 500flowers/500flowers/image_0202.jpg, dominant color HSV range: 82.5
Image: 500flowers/500flowers/image_0203.jpg, dominant color HSV range: 32.22222222222222
Image: 500flowers/500flowers/image_0204.jpg, dominant color HSV range: 42.666666666666664
Image: 500flowers/500flowers/image_0205.jpg, dominant color HSV range: 74.57142857142857
Image: 500flowers/500flowers/image_0206.jpg, dominant color HSV range: 43.37837837837838
Image: 500flowers/500flowers/image_0207.jpg, dominant color HSV range: 30.0
Image: 500flowers/500flowers/image_0208.jpg, dominant color HSV range:

Image: 500flowers/500flowers/image_0298.jpg, dominant color HSV range: 25.0
Image: 500flowers/500flowers/image_0299.jpg, dominant color HSV range: 66.73469387755102
Image: 500flowers/500flowers/image_0300.jpg, dominant color HSV range: 116.75675675675676
Image: 500flowers/500flowers/image_0301.jpg, dominant color HSV range: 53.4375
Image: 500flowers/500flowers/image_0302.jpg, dominant color HSV range: 120.66666666666667
Image: 500flowers/500flowers/image_0303.jpg, dominant color HSV range: 120.0
Image: 500flowers/500flowers/image_0304.jpg, dominant color HSV range: 48.387096774193544
Image: 500flowers/500flowers/image_0305.jpg, dominant color HSV range: 45.0
Image: 500flowers/500flowers/image_0306.jpg, dominant color HSV range: 54.0
Image: 500flowers/500flowers/image_0307.jpg, dominant color HSV range: 43.33333333333334
Image: 500flowers/500flowers/image_0308.jpg, dominant color HSV range: 88.5
Image: 500flowers/500flowers/image_0309.jpg, dominant color HSV range: 42.41379310344827
Ima

Image: 500flowers/500flowers/image_0405.jpg, dominant color HSV range: 36.70588235294118
Image: 500flowers/500flowers/image_0406.jpg, dominant color HSV range: 41.785714285714285
Image: 500flowers/500flowers/image_0407.jpg, dominant color HSV range: 129.375
Image: 500flowers/500flowers/image_0408.jpg, dominant color HSV range: 30.416666666666668
Image: 500flowers/500flowers/image_0409.jpg, dominant color HSV range: 37.358490566037744
Image: 500flowers/500flowers/image_0410.jpg, dominant color HSV range: 50.30769230769232
Image: 500flowers/500flowers/image_0411.jpg, dominant color HSV range: 24.23076923076923
Image: 500flowers/500flowers/image_0412.jpg, dominant color HSV range: 128.2758620689655
Image: 500flowers/500flowers/image_0413.jpg, dominant color HSV range: 174.0
Image: 500flowers/500flowers/image_0414.jpg, dominant color HSV range: 30.499999999999996
Image: 500flowers/500flowers/image_0415.jpg, dominant color HSV range: 49.41176470588235
Image: 500flowers/500flowers/image_0416